In [44]:
import numpy as np
import torch
import torch
from egnn_pytorch import EGNN_Network

import torch.nn as nn
import numpy as np
import random
import torch.nn.functional as F
import itertools

import torch
from matplotlib import pyplot as plt

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import dgl
from dgl.nn import GraphConv
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

charges = {'HIS': 1, 'LYS': 1,
           'ASP': -1, 'GLU': -1,  # carboxylate
           'CYS': 0, 'CYS': 0,
           'TYR': 0}   # thiol and phenol prot

cations = {'HIS': ("HD1", "HD2", "HE1", "HE2"),
        'ASP':None,
        "LYS":("HZ1", "HZ2","HZ3"),
        "TYR":"HH",
        "GLU":None,
        "CYS":'HG',
        "ARG": ("HE11","HE12", "HE21", "HE22"),
        "THR":"HG1",
        "SER":"HG",
        "TRP":"HE1"}


anions = {"HIS":("ND1", "ND2"),
          "ASP":("OD1","OD2"),
          "LYS":"NZ",
          "TYR":"OH",
          "GLU":("OE1", "OE2"),
          "CYS":"SG",
          "ARG": ("NE1","NE2"),
          "THR":"OG1",
          "SER":"OG",
          "TRP":"NE1"}

labels = {"HIS":6.08,
          "ASP":3.9,
          "LYS":10.5,
          "TYR":10.1,
          "GLU":4.3,
          "CYS":8.28}



prot = charges.keys()

Rs, solvents = {}, {}
counter = 0
all_ppos, all_pspecies = {}, {}
positions = []
s={}
all_ions={}

###############################
resi_keys=tuple(data[pdb].keys())

pspecies={}
for res in resi_keys:
    pspecies, Rs = {},{}
    residict = resis[res]
    for atom, pos in residict.items():
        node = atom.split()  # name, pos
        #rint(node)
        nodei = int(node[1])
        pspecies.update({node[0] : nodei})
        #rint(nodei)
        #print(i)
        #i += 1
        #rint({node[0] : nodei}, pos)
        Rs[nodei] = pos


    all_ppos[res] = Rs  # ({res: {node[1]: pos}})
    all_pspecies[res] = pspecies



for key in resi_keys:
    
    resname = key[:3]
    species = all_pspecies[key]
    ion = torch.zeros(len(species))


    counter1, counter2 =0, 0
    species_map={}
    for k, v in species.items():
        
        element = k[0]
        #print(element)
        if element not in species_map:
            species_map[element] = counter1
            counter1 +=1
        species[k] = counter2
        counter2 += 1
 
    cats =cations[resname]
    if cats:
        cs = [c for c in cats if c is not None]
        cats =[species.get(a) for a in cs]
        cs = [c for c in cats if c is not None]
        ion[cs] = 1


    ans = [species.get(a) for a in anions[resname]]
    ion[ans]  =-1
    
    all_ions[key] = ion

    feats = np.where(np.array(list(species.keys())) == species_map['H'], torch.tensor(1), torch.tensor(0))
    #feats = torch.tensor(feats).unsqueeze(0)


#turn species into numbers
#get pos tensor

els = list(species.keys())
es = [a[0] for a in els]

final=[]

fs = [a for a in es if a == "H"]
for a in es:
    if a=="H":
        final.append(1)
    else:
        final.append(0)

Hs = torch.tensor(final)



NameError: name 'resis' is not defined

model 1

In [ ]:
def boltzmax(x, temp):
    """
    Computes Boltzmann probabilities (Euclidean Softmax) for a tensor of distances.
    
    Parameters:
        distances (torch.Tensor): feature tensor
        temperature (float): The temperature parameter to control the sharpness.
    
    Returns:
        torch.Tensor: A tensor of notprobabilities 
    """
    # Scale the distances by temperature
    scaled_distances = x / temp
    
    # Exponentiate the scaled distances
    x = torch.exp(scaled_distances)
    p = x / torch.max(x) #doing max instead of 
    
    return p


def contrastive_loss(latent_features, species):
    """seperates latent space by enforcing dissimilarity between negative and positive ions"""

    loss = 0.0

    positive_pairs = [(i, j) for i, j in itertools.combinations(range(len(species)), 2) if species[i] == species[j]]
    negative_pairs = [(i, j) for i, j in itertools.combinations(range(3), 2) if species[i] != species[j]]


    # Positive pairs
    for i, j in positive_pairs:
        z_i, z_j = latent_features[i], latent_features[j]
        dist = torch.norm(z_i - z_j, p=2)  # Euclidean distance
        loss += dist ** 2  # Minimize distance for positive pairs

    # Negative pairs
    for i, j in negative_pairs:
        z_i, z_j = latent_features[i], latent_features[j]
        dist = torch.norm(z_i - z_j, p=2) #seperation
        loss += torch.clamp(dist, min=0) ** 2 #enforce sepertion



    return loss / (len(positive_pairs) + len(negative_pairs))



def model(num_nodes, dim, depth, lr, weight_decay):
    net = EGNN_Network(
        num_tokens = 100, #vocabulary siye, number of unique species
        num_positions = num_nodes,  #number of nodes         # unless what you are passing in is an unordered set, set this to the maximum sequence length
        dim = dim,# #internal rep size. c has square dependence. richer rep but overfitting for small d.s.
        depth = depth, #number of layers #deeper need more memort to store intermediate reps
        num_nearest_neighbors = 1, #number of nearest neighbors to consider #make this the max hood size
        coor_weights_clamp_value = 2   # absolute clamped value for the coordinate weights, needed if you increase the num neareest neighbors
    )

    optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
    return net, optimizer



def loop(nepochs, coors, species, ion_labels, model, optimizer, negative_slope):

    es = [a[0] for a in species]
    final=[]
    for a in es:
        if a=="H":
            final.append(1)
        else:
            final.append(0)
    Hs = torch.tensor(final)

    for epoch in range(200):

        optimizer.zero_grad()
        x, _ = model(Hs.unsqueeze(0), coors.unsqueeze(0)) #can turn off return po
        rep = boltzmax((nn.LeakyReLU(negative_slope=negative_slope)(x)), 15)[0] #apply activation (output raw numbers), and softmax (output nonnormalized probabilities)  
        L = contrastive_loss(rep, ion_labels)
        L.backward()
        optimizer.step()

    print(L)    
    return rep



run model 2

In [ ]:


maxes, Ns, scs=[],[],[]



#for i in range(1):
for sample in all_ppos.keys():
    if sample[:3] != "ARG":
        coords = torch.tensor(list(all_ppos[sample].values()))
        species = (list(all_pspecies[sample]))
        net, optimizer = model(len(species), 3, 2, .01, .01) # dim, depth, lr, weight decay


        x = loop(10, coords, species, all_ions[sample], net, optimizer, 1) #margin, negative slope
        out = torch.mean(x, dim=1)
        f = out.unsqueeze(0).detach().numpy()*coords.detach().numpy().T
        combined = np.column_stack(f).flatten()
        
        D=np.gradient(np.gradient(combined)).reshape(-1,3)
        #plot(x[0], D)
        print(sample)
        #plt.show()
        
        def loop2(D, sample):

            r = sample[:3]
            #out = torch.mean(x, dim=1)
            pnet = EGNN_Network(
            num_tokens = 6, #vocabulary siye, number of unique species
            num_positions = len(D),  #number of nodes         # unless what you are passing in is an unordered set, set this to the maximum sequence length
            dim = 1,# #internal rep size. c has square dependence. richer rep but overfitting for small d.s.
            depth = 2, #number of layers #deeper need more memort to store intermediate reps
            num_nearest_neighbors = 2, #number of nearest neighbors to consider #make this the max hood size
            dropout=.03,
            m_pool_method='mean')


            loss = torch.nn.HuberLoss()
            
            optimizer3 = torch.optim.Adam(pnet.parameters(), lr=.1, weight_decay=.1)
            

            ins = torch.ones(len(D))
            ins[0]=0
            ins[-1] = 2#,10,10

            

            for epoch in range(200):
                optimizer3.zero_grad()
                y=pnet(torch.tensor(ins, dtype=int).unsqueeze(0), torch.tensor(D).unsqueeze(0))[0]
                label=labels[r]
                loss2=loss(max(y[0][1::]), torch.tensor(label))
                loss2.backward()
                optimizer3.step()

            a=y[0].detach().numpy()
            maxes.append((np.where(a == max(a[1::]))[0], max(a[1::])))
            Ns.append(a[0])
            scs.append(a[-1])


        return maxes, Ns, scs
    m, n, scs = loop2(D, sample)
        

In [ ]:
full_dataset = torch.tensor(list(list(all_ppos.values())[0].values()), dtype=torch.float32)

In [39]:


def load_pdb(path):
    with open(path, "r") as f:
        resis = {}
        xs = []
        counter=0
        for line in f:
            L = line.split()

            if line.startswith("ATOM"):
                resname, atomname = L[3], L[2]
                if resname in prot:
                    resname = resname + str(L[5])

                    if resname in resis:
                        resis[resname][f"{atomname} {counter}"] = (float(L[6]), float(L[7]), float(L[8]))
                        counter += 1
                    else:
                        resis[resname] = {f"{atomname} {counter}": (float(L[6]), float(L[7]), float(L[8]))}
                        counter += 1

                else:  # resname in noprot
                    resname, atomname, resnum = L[3], L[2], L[5]
                    xs.append((float(L[6]), float(L[7]), float(L[8])))
                    counter += 1

            elif line.startswith("HETATM"):
                resname = L[3] + str(L[5])
                atomname = L[2]
                if resname in solvents:
                    solvents[resname][f"{atomname} {str(counter)}"] = (float(L[6]), float(L[7]), float(L[8]))
                    counter += 1
                else:
                    solvents[resname] = {f"{atomname} {str(counter)}": (float(L[6]), float(L[7]), float(L[8]))}
                counter += 1
    #print(resis)
    return resis

#load_pdb()
import glob
import dask.dataframe as dd

# Specify the directory and pattern
directory_path = '/home/jrhoernschemeyer/Desktop/thesis/PDB/*red.pdb'  # Replace with your directory path

# Retrieve all .txt file paths
pdbs=[]
data={}
files = glob.glob(directory_path)
for file in files:
    pdb = file[-12:-8]
    data[pdb] = load_pdb(file)
    pdbs.append(pdb)
    #data[pdb]
#data = [load_pdb(file) for file in files]
#pdbs = [file[-12:-8] for file in files]
#set(list(data.keys()))
#dic = {}
#datas=[]
#for p,d in zip(pdbs, data):
    #entry = 
    #dic[p] = d 
    
    #datas.append(list(d))
#print(datas)
#print(dic)
    


def read_database(path):
    """csv --> dask df"""
    #make the dask data frame from the PYPKA csv
    dk=dd.read_csv(path, sep=';')
    return dk#.reset_index()



    
#df = read_database("/home/jrhoernschemeyer/Desktop/thesis/pkas.csv")
#def (pdbs, df):
all_targets={}
for pdb in pdbs:
    2
    #df = pkpdb[pkpdb.iloc[:, 0] == pdb].drop(columns = ["idcode"])
    all_targets[pdb] = torch.tensor(list(df["pk"]))


#print(df)

In [48]:
for pdb in pdbs:
    resi_keys=tuple(data[pdb].keys())
    pspecies={}
    all_resis = data[pdb]
    for res in resi_keys:
        print(res)

TYR3
TYR7
CYS14
ASP23
LYS29
GLU30
GLU31
GLU36
GLU40
LYS44
CYS47
TYR49
LYS54
ASP57
ASP59
TYR63
HIS71
TYR79
LYS81
ASP82
GLU85
ASP90
ASP94
GLU97
ASP98
CYS101
LYS102
TYR103
TYR108
TYR111
GLU112
LYS115
ASP116
ASP117
TYR118
LYS120
LYS127
GLU130
LYS140
ASP146
ASP152
TYR153
ASP157
HIS162
GLU163
CYS169
ASP171
TYR179
LYS188
LYS190
GLU197
TYR198
LYS208
GLU5
ASP10
GLU11
LYS16
TYR18
LYS19
ASP20
GLU22
TYR24
TYR25
HIS31
LYS35
LYS43
LYS44
GLU45
ASP47
LYS48
LYS60
ASP61
GLU62
GLU64
LYS65
ASP70
ASP72
LYS83
LYS85
TYR88
ASP89
ASP92
GLU108
LYS124
ASP127
GLU128
LYS135
TYR139
LYS147
ASP159
TYR161
LYS162
GLU5
GLU7
HIS13
LYS17
GLU19
ASP21
HIS25
ASP28
LYS35
HIS37
GLU39
GLU42
LYS43
ASP45
LYS48
HIS49
LYS51
GLU53
GLU55
LYS57
GLU60
ASP61
LYS63
LYS64
HIS65
LYS78
LYS79
LYS80
HIS82
HIS83
GLU84
GLU86
LYS88
HIS94
LYS97
HIS98
LYS99
LYS103
TYR104
GLU106
GLU110
HIS114
HIS117
HIS120
ASP127
LYS134
GLU137
LYS141
ASP142
LYS146
TYR147
LYS148
GLU149
TYR152
GLU5
GLU7
HIS13
LYS17
GLU19
ASP21
HIS25
ASP28
LYS35
HIS37
GLU39
GLU42
LYS4

In [261]:
all_pos = {}
counter=0
all_species={}
all_ions={}
Rs={}
all_lengths = {}

for pdb in pdbs[2:]:
    
    

    resis = data[pdb] #for one protein
    print(pdb)

    #resi_keys=tuple(data[pdb].keys())
    resi_pos ={}
    lengths={}
    
    
    for resid, vals in resis.items():
        counter=0
        resi_pos, resi_species ={},{}#,{}
        #print(resid)
        #ion = torch.zeros(len())
        for atom, pos in vals.items():
            #print(counter, atom)
            #rint(counter)
            #print(2, pos)#, pos)

            node = atom.split()  # name, pos
            #print(node)
            #print(node)
            nodei = int(node[1])
            #pspecies.update({node[0] : nodei})
            #rint(nodei)
            #print(i)
            #i += 1
            #rint({node[0] : nodei}, pos)
            #print(str(counter))
            resi_pos[counter] = pos
            
            #print(counter)
            #print(node[0])
            resi_species[node[0]] = counter
            
            counter +=1

            #print(3, resi_species)


            #resi_elements = 
            #nodei = int(node[1])
            #pspecies.update({node[0] : nodei})
        #elements = [e[0] for e in resi_species.values()]
        







    all_pos[pdb]=Rs
    all_species[pdb] = resi_species
    all_lengths[pdb] = lengths




111m
110m
108m
110l
109m
111l
108l
104m
106m
105m
103m
107m
107l
103l
102m
102l
101m
104l


In [260]:
resi_species

{'N': 22,
 'H': 23,
 'CA': 24,
 'HA': 25,
 'C': 26,
 'O': 27,
 'CB': 28,
 'HB2': 29,
 'HB3': 30,
 'CG': 31,
 'HG2': 32,
 'HG3': 33,
 'CD': 34,
 'HD2': 35,
 'HD3': 36,
 'CE': 37,
 'HE2': 38,
 'HE3': 39,
 'NZ': 40,
 'HZ1': 41,
 'HZ2': 42,
 'HZ3': 43}

In [246]:
resi_ions={}
pdb_ions={}
def ionic_features(pdbs):
    all_ions={}
    for pdb in pdbs:
        pdb_ions={}
        Ls = all_lengths[pdb]
        
        for resi in data[pdb].keys():
            
            resi_species = (all_species[pdb])[resi]

            ion = torch.zeros(Ls[resi])

            resname = resi[:3]

            idxs = [resi_species.get(a) for a in anions[resname]] 
            ion[idxs]= -1
            
            cats = cations[resname]
            if cats:
                cs = [c for c in cats if c is not None]
                cats =[resi_species.get(a) for a in cs]
                cs = [c for c in cats if c is not None]
                ion[cs] = 1
            pdb_ions[resi] = ion
        all_ions[pdb] = pdb_ions
    
    
    return all_ions

def get_H_indices(pdbs):
    for pdb in pdbs[:2]:
        
        #print(all_species[pdb].values())
        for resid, vals in all_species[pdb].items():
                print(vals)
                #resi_pos, resi_species, ions ={},{},{}
                #ion = torch.zeros(len())
                #for atom, pos in vals.items():




    
all_ions = ionic_features(pdbs)

In [249]:
all_species

{'10gs': {'GLU5': {'N': 15,
   'H': 16,
   'CA': 17,
   'HA': 18,
   'C': 19,
   'O': 20,
   'CB': 21,
   'HB2': 22,
   'HB3': 23,
   'CG': 24,
   'HG2': 25,
   'HG3': 26,
   'CD': 27,
   'OE1': 28,
   'OE2': 29},
  'ASP10': {'N': 12,
   'H': 13,
   'CA': 14,
   'HA': 15,
   'C': 16,
   'O': 17,
   'CB': 18,
   'HB2': 19,
   'HB3': 20,
   'CG': 21,
   'OD1': 22,
   'OD2': 23},
  'GLU11': {'N': 15,
   'H': 16,
   'CA': 17,
   'HA': 18,
   'C': 19,
   'O': 20,
   'CB': 21,
   'HB2': 22,
   'HB3': 23,
   'CG': 24,
   'HG2': 25,
   'HG3': 26,
   'CD': 27,
   'OE1': 28,
   'OE2': 29},
  'LYS16': {'N': 22,
   'H': 23,
   'CA': 24,
   'HA': 25,
   'C': 26,
   'O': 27,
   'CB': 28,
   'HB2': 29,
   'HB3': 30,
   'CG': 31,
   'HG2': 32,
   'HG3': 33,
   'CD': 34,
   'HD2': 35,
   'HD3': 36,
   'CE': 37,
   'HE2': 38,
   'HE3': 39,
   'NZ': 40,
   'HZ1': 41,
   'HZ2': 42,
   'HZ3': 43},
  'TYR18': {'N': 21,
   'H': 22,
   'CA': 23,
   'HA': 24,
   'C': 25,
   'O': 26,
   'CB': 27,
   'HB2': 28,


In [247]:
get_H_indices(pdbs)

{'N': 21, 'H': 22, 'CA': 23, 'HA': 24, 'C': 25, 'O': 26, 'CB': 27, 'HB2': 28, 'HB3': 29, 'CG': 30, 'CD1': 31, 'HD1': 32, 'CD2': 33, 'HD2': 34, 'CE1': 35, 'HE1': 36, 'CE2': 37, 'HE2': 38, 'CZ': 39, 'OH': 40, 'HH': 41}
{'N': 21, 'H': 22, 'CA': 23, 'HA': 24, 'C': 25, 'O': 26, 'CB': 27, 'HB2': 28, 'HB3': 29, 'CG': 30, 'CD1': 31, 'HD1': 32, 'CD2': 33, 'HD2': 34, 'CE1': 35, 'HE1': 36, 'CE2': 37, 'HE2': 38, 'CZ': 39, 'OH': 40, 'HH': 41}
{'N': 11, 'H': 12, 'CA': 13, 'HA': 14, 'C': 15, 'O': 16, 'CB': 17, 'HB2': 18, 'HB3': 19, 'SG': 20, 'HG': 21}
{'N': 12, 'H': 13, 'CA': 14, 'HA': 15, 'C': 16, 'O': 17, 'CB': 18, 'HB2': 19, 'HB3': 20, 'CG': 21, 'OD1': 22, 'OD2': 23}
{'N': 22, 'H': 23, 'CA': 24, 'HA': 25, 'C': 26, 'O': 27, 'CB': 28, 'HB2': 29, 'HB3': 30, 'CG': 31, 'HG2': 32, 'HG3': 33, 'CD': 34, 'HD2': 35, 'HD3': 36, 'CE': 37, 'HE2': 38, 'HE3': 39, 'NZ': 40, 'HZ1': 41, 'HZ2': 42, 'HZ3': 43}
{'N': 15, 'H': 16, 'CA': 17, 'HA': 18, 'C': 19, 'O': 20, 'CB': 21, 'HB2': 22, 'HB3': 23, 'CG': 24, 'HG2': 25

In [131]:
all_targets
all_pos
all_species

20


In [238]:
species_map

NameError: name 'species_map' is not defined

In [46]:
for pdb in pdbs:
    
    pspecies={}
    resi_dict = data[pdb] #for one protein
    #resi_keys=tuple(data[pdb].keys())
    for atom, pos in residict.items():
                node = atom.split()  # name, pos
                #rint(node)
                nodei = int(node[1])
                pspecies.update({node[0] : nodei})
                #rint(nodei)
                #print(i)
                #i += 1
                #rint({node[0] : nodei}, pos)
                Rs[nodei] = pos
    for res in resi_keys:
        2
        #all_
        pspecies, Rs = {},{}
        residict 
        for atom, pos in residict.items():
            node = atom.split()  # name, pos
            #rint(node)
            nodei = int(node[1])
            pspecies.update({node[0] : nodei})
            #rint(nodei)
            #print(i)
            #i += 1
            #rint({node[0] : nodei}, pos)
            Rs[nodei] = pos


        all_ppos[res] = Rs  # ({res: {node[1]: pos}})
        all_pspecies[res] = pspecies



    for key in resi_keys:
        
        resname = key[:3]
        species = all_pspecies[key]
        ion = torch.zeros(len(species))


        counter1, counter2 =0, 0
        species_map={}
        for k, v in species.items():
            
            element = k[0]
            #print(element)
            if element not in species_map:
                species_map[element] = counter1
                counter1 +=1
            species[k] = counter2
            counter2 += 1
    
        cats =cations[resname]
        if cats:
            cs = [c for c in cats if c is not None]
            cats =[species.get(a) for a in cs]
            cs = [c for c in cats if c is not None]
            ion[cs] = 1


        ans = [species.get(a) for a in anions[resname]]
        ion[ans]  =-1
        
        all_ions[key] = ion

        feats = np.where(np.array(list(species.keys())) == species_map['H'], torch.tensor(1), torch.tensor(0))
        #feats = torch.tensor(feats).unsqueeze(0)



    

TypeError: tuple indices must be integers or slices, not str

In [41]:
#prot = charges.keys()

Rs, solvents = {}, {}
counter = 0
all_ppos, all_pspecies = {}, {}
positions = []
s={}
all_ions={}

###############################
resi_keys=tuple(data[pdb].keys())

pspecies={}
for res in resi_keys:
    pspecies, Rs = {},{}
    residict = resis[res]
    for atom, pos in residict.items():
        node = atom.split()  # name, pos
        #rint(node)
        nodei = int(node[1])
        pspecies.update({node[0] : nodei})
        #rint(nodei)
        #print(i)
        #i += 1
        #rint({node[0] : nodei}, pos)
        Rs[nodei] = pos


    all_ppos[res] = Rs  # ({res: {node[1]: pos}})
    all_pspecies[res] = pspecies



for pdb in pdbs:
    struc_targets = all_targets[pdb] 
    #struc_pos = 

tensor([10.7704, 10.9351,  9.2148,  9.7232, 11.2614, 10.6484,  2.2228,  3.6045,
         3.8890,  3.3866,  9.7061,  2.4660,  9.5755, 10.4136, 10.8397,  2.0361,
         8.1290,  3.6044,  3.1161, 10.0845,  1.2090, 10.4125,  3.0962, 11.2576,
         3.5727,  2.5252,  9.7423, 11.5582, 10.2297,  3.9142, 11.3792,  1.9389,
         2.8609,  2.8246, 11.1933,  3.5675, 15.2945,  0.3837, 10.4133,  3.1562,
         3.7790,  9.9409,  2.5118,  7.6023,  2.4467,  1.8037, 10.3570,  3.1635,
         2.0262,  3.1815,  3.5454, 10.7322,  4.0791, 11.0590,  4.0168, 10.3651,
         3.4528, 10.9320,  1.4520, 13.1578,  7.9018,  2.5366, 10.7460, 11.0298,
         3.5499, 10.9687,  3.2963, 11.3751, 10.5962, 10.3610,  7.7762,  3.4684,
        10.7966,  3.3960, 11.2644, 10.3201,  2.9648, 12.9127,  7.6161,  4.1005])

In [42]:
all_ppos

{}

In [10]:
df = read_database("/home/jrhoernschemeyer/Desktop/thesis/pkas.csv")
pkpdb = df.compute()

final model, predict pkas on batched split data

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn split
X_train, X_val, y_train, y_val = train_test_split(
    full_dataset, labels, test_size=0.2, random_state=42, stratify=labels
)

# Convert to PyTorch tensors
train_tensor = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_tensor = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))

# PyTorch DataLoader
train_loader = DataLoader(
    train_tensor,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_tensor,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


#val_loader.dataset.tensors
#train_loader.dataset.tensors

In [ ]:
targets=[3.49973,
1.13269,
4.32236,
10.366,
11.1131,
11.2963,
2.78064,
2.79104,
10.3907,
9.83911,
7.78241,
10.9913,
9.36903,
3.27824,
2.62777,
11.4944,
10.5661,
3.60359,
2.22548,
4.09207,
10.8503,
1.76499,
3.03049,
10.377,
11.2809,
10.1192,
2.5044,
0.977022,
3.33801,
10.0178,
3.46841,
3.48914,
10.2195,
9.58874,
11.1147,
2.95551,
14.4481,
10.9523]

scindex=[]
for id in resis.keys():
    scindex.append(len(resis[id]))

N = [np.array((0, v.item())) for v in n]
S = [np.array((i, v.item())) for i,v in zip(scindex, scs)]

    
B = [np.concatenate((i[0], i[1])) for i in m] #maxes, is in the backbone
#len(resis["GLU5"])

#np.vstack([N,S,B]).reshape(-1,2)

triangles=[]
for i in range(len(N)):
    triangles.append(np.array([N[i], S[i], B[i]]))
    


for k, v in all_pspecies.items():
    resi, val = k[0], k[3:]
    #ens[k[0]]  len(v)
    print(resi,val)



def create_residue_graph(sequence, coordinates, distance_threshold=None):
    """
    Creates a DGL graph for a single residue with 3 nodes.

    Parameters:
    - sequence (list): List of residues (single-letter codes). Should correspond to 3 nodes.
    - coordinates (np.array or torch.Tensor): Shape (3, 3), 3D coordinates of nodes.
    - distance_threshold (float): Distance to consider creating an edge.

    Returns:
    - DGLGraph
    """
    num_nodes = 3  # Each graph has 3 nodes
    g = dgl.graph(([], []), num_nodes=num_nodes)

    # Add edges based on sequential adjacency
    src = [0, 1, 2]
    dst = [1, 2, 0]
    g.add_edges(src, dst)
    g.add_edges(dst, src)  # Make it bidirectional

    # Optionally add edges based on spatial proximity
    if distance_threshold is not None and coordinates is not None:
        if isinstance(coordinates, torch.Tensor):
            coordinates = coordinates.numpy()
        
        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                dist = np.linalg.norm(coordinates[i] - coordinates[j])
                if dist <= distance_threshold:
                    g.add_edges([i], [j])
                    g.add_edges([j], [i])

    return g

graphs = []
features = []
all_coords = []

for residue_index in range(38):  # Assuming 48 residues
    # Example: Get coordinates for 3 nodes of the current residue
    coords = tpos[residue_index]  # Replace with your actual data source
    #print(coords.shape)
    residue = sequence[residue_index]  # Residue identifier (single-letter code)
    #print(residue)
    # Create a graph for this residue
    g = create_residue_graph([residue] * 3, coords, distance_threshold=5.0)
    print(g)
    #g = dgl.add_self_loop(g)  # Add self-loops
    
    graphs.append(g)

    # Use coordinates as features for the 3 nodes
    #node_features = torch.tensor(coords, dtype=torch.float32)  # Shape: (3, 3)
    features.append(torch.tensor(triangles[residue_index],dtype=torch.float32))
    #print(len(features[0]))
    # Save coordinates for debugging
    all_coords.append(coords)


def create_protein_graph(residue_graphs, residue_positions, distance_threshold):
    """
    Creates a protein-level graph with residue-level information.

    Parameters:
    - residue_graphs (list): List of 48 residue-level graphs (DGLGraphs).
    - residue_positions (torch.Tensor): Shape (48, 3), 3D positions for each residue.
    - distance_threshold (float): Distance threshold for creating edges.

    Returns:
    - protein_graph (DGLGraph): Protein-level graph with 48 nodes.
    """
    num_residues = len(residue_graphs)
    g = dgl.graph(([], []), num_nodes=num_residues)

    # Add edges based on pairwise distances
    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            dist = torch.norm(residue_positions[i] - residue_positions[j])
            if dist <= distance_threshold:
                g.add_edges(i, j)
                g.add_edges(j, i)  # Make it bidirectional

    return g


features_list = [f[-1][1] for f in features]
#features_list

poslist=[]
tpos=[]
for a in all_ppos.keys():
    poslist.append(list(all_ppos[a].values()))

idxs=[]  
for p,b in zip(poslist, triangles):
    idxs.append([y[0] for y in b])
for i, j in zip(idxs, poslist):

    idxs = ([int(k) for k in i])
    m=idxs[-1]
    tpos.append(np.array([j[0], j[-1], j[m]]))
print(torch.tensor(tpos))

residue_positions = [p[-1] for p in tpos]
#residue_positions

# Create the protein-level graph
protein_graph = create_protein_graph(graphs, torch.tensor(residue_positions), distance_threshold=20.0)

# Add features to the protein-level graph
protein_features = [features.flatten() for features in features_list]  # Flatten (3, 2) -> (6,)
protein_features = torch.stack(protein_features)  # Shape: (48, 6)
protein_graph.ndata['features'] = protein_features

class MultiresolutionModel(nn.Module):
    def __init__(self, residue_in_feats, residue_hidden_feats, protein_hidden_feats, out_feats):
        super(MultiresolutionModel, self).__init__()
        

        # Final output layer
        self.fc = nn.Linear(38,38)

    def forward(self, residue_graphs, residue_features, protein_graph):
        # Step 1: Process residue-level graphs
        residue_embeddings = []
        for g, features in zip(residue_graphs, residue_features):
            residue_embeddings.append(features[2][1])
        residue_embeddings = torch.stack(residue_embeddings)  # Shape: (48, residue_hidden_feats)
        protein_graph.ndata['features'] = residue_embeddings.unsqueeze(1)
        
        outputs = self.fc(residue_embeddings)  # Shape: (48, out_feats)

        return outputs

model = MultiresolutionModel(2, 1, 1,38)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=.01)
criterion = nn.HuberLoss()

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(graphs, features, protein_graph)  # Shape: (48, out_feats)
    #print(outputs[:3])
    #print(outputs)
    # Compute loss (per residue)
    print(outputs)
    loss = criterion(outputs, torch.tensor(targets))  # Ensure residue_targets is shape (48, out_feats)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")


#plt.plot(outputs.detach().numpy())

#plt.plot(outputs.detach().numpy())




TypeError: 'int' object is not iterable